# Beginnings - Framing the Game

- decided on a box layout with a horizontal orientation splitting the screen in two
- wishlist: dynamically switching between horizontal & vertical box layouts (for something like mobile)
- TODO: even split at this point, will offset with more size toward the board later
- put in a few different boxes on the right for different game context as placeholder values

# Board Rendering - First Try!

- Looked at how we were going to render the board so that you could click on intersections to play them
- Setup a simple GridLayout to be buttons for clicking, just a small 3x3 to start
- Tried to set a background image on that layout, but weren't having success with that
- Kjell suggested we use a Float Layout, so that we could just have the GridLayout & an Image be absolutely positioned on top of each other, worked!
- Had a while trying to figure out how to make the GridLayout fit the image in particular rather than the entire entry of the parent BoxLayout
    - The image itself keeps its square regardless of how the screen is resized, we need Buttons to follow that
    - turned out that the trick was using Image's [norm_image_size](https://kivy.org/docs/api-kivy.uix.image.html?highlight=image#kivy.uix.image.Image.norm_image_size) to get the size of the image, no matter how it was resized
    - Once we had this, the image was the right size, but not in the right position. We noticed that what we wanted from the position was to be in the middle of the frame, and Kjell immediately informed us that `pos_hint: 0.5, 0.5` was the magic we were looking for
- BEHOLD!

In [36]:
%%html
<img src="Screen Shots/df4961a.png" width=1000 />

# Expanding the Button Grid

- Goal: Expand from the 3x3 board to the 361 buttons needed to cover the full 19x19 board
- Didn't seem a good idea to add 361 children to a .kv file, so we looked into dynamically generating them
- Kjell gave us some code for resizing the generated buttons based on their parent's size (as it is resized)
    - we aren't dealing with piece images yet. we'll see the benefits of that later
- Turned out adding buttons to the GridLayout do get put in the correct spot thanks to the layout parameters
- Did some tweaking of the padding & spacing parameters on GridLayout until satisfied with deadzones between the playable intersections

In [37]:
%%html
<img src="Screen Shots/361 buttons positioned correctly.png" width=1000 />

# The Wee Hours of the Night

- Andrew went on a solo journey after dinner, when Kevin departed from the group
- Made the changeover from notebook development to individual file development:
    - Brought the notebook completely up-to-date
    - Had the notebook create the files one last time
    - Cut over to developing the files that the notebook used to generate
    - Deleted the notebook from the repo
- Andrew brought out his secret weapon, an older go module that he'd previously made to manage board state (`go.py`)

# Morning, the 2nd

- Tried out running the game without debug mode, made sure it presents correctly
- Wired together button `on_press` events to button info, mainly index
    - You can see our button clicks have reported their intersection index in the console log below:

In [38]:
%%html
<img src="Screen Shots/No debug mode - intersection IDs set.png" width=1000 />

# Wiring Together the Go Representation & Rendering

- Created a bind callback for our intersection buttons that reports click positions back to the go representation library
    - go library has nice functionality to print out the board state of a position in the console
- during this process, we ran into [Common Problem 1](#commonproblem1). Check the link for more info
- Setup representation & pass through to board, so that positions could be drawn correctly from the representation
    - initially, just setting the button text to the appropriate letter ('', 'B', or 'W') before moving to images
- Added images (check [Assets](#Assets) section for more info) for the stones, & hooked them in instead of the text
    - Actually created a descendant of Image called StoneImage
    - Has its own `sourceblack` & `sourcewhite` properties, which point to the different image sources
    - When game state changes, we change the image source to be one of the two above
    - We also set the color transparency, so that you can have an intersection empty as well
        - This happens in the board start position, & also after captures
- Now that all of this happens, a particular board state is shown below:

In [39]:
%%html
<img src="Screen Shots/508dbfcc.png" width=1000 />

# Error Codes & Bad Play

- In The Beginning, the go module throws exceptions for illegal play
- First move: just ignore the exceptions
    - Since the UI only reads from the state, & the state doesn't update for illegal play, illegal moves don't change the board
    - Unfortunately, they also don't update you in any way to let you know you've made an illegal move
- Next: Passing the error message (from the go library) to the Context window on the right side of the screen
    - This looked pretty tricky to us, as we'd have to pass the message from buttons (at the bottom of the board tree inside GridLayout) to the Context buttons (at the opposite end of the board tree in the context window)
    - Jacob helped us out by teaching us [trick 1](#trick1), listed below
    - This allowed us to set text from one to the other. An error situation is shown below:

In [40]:
%%html
<img src="Screen Shots/f7f99a321.png" width=1000 />

# Day 2, Fin

- Coffee Count: 3

<a id="commonproblem1"></a>
# Common Problem 1: Races

In kivy, there is a common problem (especially in `__init__` methods) where you can try to call some kivy functionality before it's ready. In this case, what's usually thrown is an `AttributeError`. Usually in kivy, there isn't much you can do to make sure you execute your code after other things are ready.

In kivent, there's much more code around this process, and this issue can be managed through the tools they provide. But since we're just living without that at this point, we'll resort to the workaround, listed here:

In [41]:
# commented out so this doesn't actually run. You'll need it for the Clock.schedule_once method below
# from kivy.clock import Clock

def code_youre_trying_to_run(self):
    try:
        self.method1()
    except AttributeError:
        Clock.schedule_once(self.method1)
    
def method1(self, instance=None, value=None):
    do_what_you_actually_wanted()

What this does is tries your method, but if it doesn't work, then it tries scheduling your method for later execution.

When does it execute? The one parameter version of `schedule_once` schedules it for the next clock tick that the process gets, but you can also provide a number as the second parameter, which will be the number of clock ticks after that that it'll execute.

Note that the `method1` method takes a specific signature, and you'll have to work around that. If you'd rather ignore those parameters, you can do what we did above (just not using them at all), or you can use the lambda method shown below if you'd like to use variables from the `try-except` context:

In [42]:
Clock.schedule_once(lambda dt: self.add_books(local_vars_go_here), 0.1)

<ClockEvent callback=<function <lambda> at 0x7f2b22713048>>

<a id="trick1"></a>
# Trick 1: Traversing Deep Trees


- If you need to traverse up and down a kivy tree a lot, a good way to get at that is to use the root app's `ids` dict:
- Caveat: This only works for items with their id defined in the main kivy App tree
- Circumstantial: Suspect that if you get at any kivy rule object, they might have their own `ids` dict
- Call `App.get_running_app().root.ids` to reference the dict
- Once you have that `ids` dict, you can reference it either through a key, or by dot notation:
    - `ids['_image_id']` or `ids._image_id` should both work
- The object that you get back is the one referenced by that particular ID, so you can immediately chain calls to the referenced object after the ID

# Caveats

- bit of a weird state to begin with, were working on different python versions (2 & 3)
    - didn't end up biting us while developing from the notebook
    - During Andrew's solo adventures, system was updated to 3, so we worked on 3 together after that

# Assets

- images
    - [board](https://upload.wikimedia.org/wikipedia/commons/thumb/9/9b/Blank_Go_board.svg/2000px-Blank_Go_board.svg.png)
    - [black stone](https://upload.wikimedia.org/wikipedia/en/thumb/b/b6/Realistic_Go_Stone.svg/250px-Realistic_Go_Stone.svg.png)
    - [white stone](https://upload.wikimedia.org/wikipedia/en/thumb/2/20/Realistic_White_Go_Stone.svg/250px-Realistic_White_Go_Stone.svg.png)
- sounds

# TODOs

- license?
- non-debug mode splash screen

# GIFs of Awesome

Be warned. Awesomeness located below:

In [43]:
%%html
<img src="Screen Shots/gokivygo.gif" width=600 />

In [44]:
%%html
<img src="Screen Shots/gokivygo_ladder01.gif" width=600 />